In [ ]:
# import tensorflow as tf
import pandas as pd
import sklearn
from datetime import datetime, timedelta

weather_df = pd.read_csv('weather_data.csv')
observations_df = pd.read_csv('observations.csv')

# convert datetime to same format as observations_df
weather_df['Datetime'] = pd.to_datetime(weather_df['Datetime'], format='%Y%m%d%H')
weather_df['Date'] = weather_df['Datetime'].dt.date

# Use same format as weather_df
observations_df['Date'] = pd.to_datetime(observations_df[['year', 'month', 'day']])

# Group by id and Date to find all unique days per ID
day_indices = weather_df.groupby(['id', 'Date']).first().reset_index()

# remove windspeed and specific humidity columns
day_indices = day_indices.drop(columns=['Wind Speed', 'Specific Humidity'])

# make dict of observation ID with original date
observation_dates = {}
for _, row in observations_df.iterrows():
    observation_dates[row['id']] = row['Date']

# Calculate difference between observation date and weather date
def get_relative_day(row):
    obs_date = observation_dates.get(row['id'])
    this_date = pd.to_datetime(row['Date'])
    difference = (this_date - obs_date).days
    return difference

# Apply the function to get relative day
day_indices['RelativeDay'] = day_indices.apply(get_relative_day, axis=1)

# Add relative day to weather_df
day_map = day_indices.set_index(['id', 'Date'])['RelativeDay']
weather_df['RelativeDay'] = weather_df.set_index(['id', 'Date']).index.map(day_map)

# Variables to calculate aggregations for
weather_vars = ['Temperature', 'Humidity','Dew/Frost Point', 
                'Wet Bulb Temperature', 'Precipitation', 'Soil Temperature']

# Time windows to aggregate over (days before and after the observation day)
time_windows = {
    'day_of': (0, 0),
    'day_before_after': (-1, 1),
    'three_days': (-3, 3),
    'one_week': (-7, 7),
    'two_weeks': (-14, 14)
}

# Function to calculate aggregates given ID and time window
def calculate_aggregates(weather_subset, var_name, window_name, start_day, end_day):
    # Filter data for given time window
    window_data = weather_subset[(weather_subset['RelativeDay'] >= start_day) & (weather_subset['RelativeDay'] <= end_day)]
    
    # Calculate aggregate and return as a series
    result = pd.Series({
        f"{var_name}_{window_name}_mean": window_data[var_name].mean(),
        f"{var_name}_{window_name}_min": window_data[var_name].min(),
        f"{var_name}_{window_name}_max": window_data[var_name].max(),
        f"{var_name}_{window_name}_median": window_data[var_name].median()
    })
    
    return result

# Create empty DataFrame to store results
all_features = []
result_rows = []

# For each observation row, calculate all aggregate weather data
for id_val in observations_df['id'].unique():
    id_weather = weather_df[weather_df['id'] == id_val]
        
    # Dictionary to store aggregated values for this ID
    id_features = {'id': id_val}
    
    # Iterate over all weather data variables
    for var in weather_vars:
        # iterate over all items in time_windows
        for window_name, (start_day, end_day) in time_windows.items():
            aggs = calculate_aggregates(id_weather, var, window_name, start_day, end_day)
            id_features.update(aggs)
            
            # Add feature names to our list (only once)
            if id_val == observations_df['id'].unique()[0]:
                all_features.extend(aggs.index.tolist())
    
    result_rows.append(id_features)

# Create DataFrame from results
aggregated_features = pd.DataFrame(result_rows)

# Merge with the original observations dataframe
observations_df = observations_df.merge(aggregated_features, on='id', how='left')

# Create list of feature names
weather_feature_names = all_features

print(f"Created {len(weather_feature_names)}  features")



# Add day of year as a feature to observations_df
observations_df['day_of_year'] = observations_df.apply(
    lambda row: datetime(row['year'], row['month'], row['day']).timetuple().tm_yday, 
    axis=1
)

# Drop rows with NaN values
observations_df = observations_df.dropna()

# Display the updated DataFrame
observations_df.head()
# Save the updated DataFrame to a new CSV file
observations_df.to_csv('observations_with_weather_features.csv', index=False)


In [ ]:
from sklearn.model_selection import train_test_split

import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random


# set random seed for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)


# Define the features and target variable
features = weather_feature_names + ['day_of_year']
target = 'PowderyMildew'

X = observations_df[features]
y = observations_df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(len(features),)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation ='sigmoid')  # Output layer
])
# Compile the model

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
# Plot training history
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()
# Save the model
model.save('powdery_mildew_model.h5')
# Load the model
loaded_model = tf.keras.models.load_model('powdery_mildew_model.h5')
# Make predictions
predictions = loaded_model.predict(X_test)
# Display predictions
predictions_df = pd.DataFrame({'Predicted': predictions.flatten(), 'Actual': y_test})
predictions_df.head()

# test and output confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
# Convert predictions to binary classification
threshold = 0.5
y_pred_binary = (predictions.flatten() > threshold).astype(int)
# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred_binary)
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
# Print classification report
print(classification_report(y_test, y_pred_binary, target_names=['No', 'Yes']))


In [ ]:
# train a decision tree classifier
import tensorflow_decision_forests as tfdf

# set random seed for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

observations_df = pd.read_csv('observations_with_weather_features.csv')
# split df into train and test
train=observations_df.sample(frac=0.8,random_state=200)
test=observations_df.drop(train.index)

train.drop(columns=['Date', 'coordinates', 'date_string'], inplace=True)
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="PowderyMildew")

test.drop(columns=['Date', 'coordinates', 'date_string'], inplace=True)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="PowderyMildew")

tuner = tfdf.tuner.RandomSearch(num_trials=50)

tuner.choice("min_examples", [2, 5, 7, 10])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

# merge=True indicates that the parameter (here "growing_strategy") is already
# defined, and that new values are added to it.
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])


# Create the model
model = tfdf.keras.RandomForestModel(verbose=2,tuner=tuner)
# Train the model
model.compile(metrics=["accuracy"])
model.fit(train_ds)
# Evaluate the model
evaluation = model.evaluate(test_ds)
print(evaluation)
# Make predictions
predictions = model.predict(test_ds)
# Convert predictions to binary classification
threshold = 0.5
y_pred_binary = (predictions.flatten() > threshold).astype(int)
# Generate confusion matrix
cm = confusion_matrix(test['PowderyMildew'], y_pred_binary)
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
# Print classification report
print(classification_report(test['PowderyMildew'], y_pred_binary, target_names=['No', 'Yes']))

# save model
model.save("powdery_mildew_model_decision_tree")




In [ ]:
# train a decision tree classifier
import tensorflow_decision_forests as tfdf

# set random seed for reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

observations_df = pd.read_csv('observations_with_weather_features.csv')
# split df into train and test
train=observations_df.sample(frac=0.8,random_state=200)
test=observations_df.drop(train.index)

train.drop(columns=['Date', 'coordinates', 'date_string'], inplace=True)
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="PowderyMildew")

test.drop(columns=['Date', 'coordinates', 'date_string'], inplace=True)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label="PowderyMildew")

# Create a Random Search tuner with 50 trials and automatic hp configuration.
tuner = tfdf.tuner.RandomSearch(num_trials=50, use_predefined_hps=True)

# Define and train the model.
model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
model.fit(train_ds, verbose=2)

# Train the model
model.compile(metrics=["accuracy"])
model.fit(train_ds)
# Evaluate the model
evaluation = model.evaluate(test_ds)
print(evaluation)
# Make predictions
predictions = model.predict(test_ds)
# Convert predictions to binary classification
threshold = 0.5
y_pred_binary = (predictions.flatten() > threshold).astype(int)
# Generate confusion matrix
cm = confusion_matrix(test['PowderyMildew'], y_pred_binary)
# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
# Print classification report
print(classification_report(test['PowderyMildew'], y_pred_binary, target_names=['No', 'Yes']))

# Save the model
model.save('gradient_boosted_model')